# Scoring Script Test
The purpose of this test is a proof of concept implementation of NannyML performance estimation and EvidentlyAI data drift calculations into the bridging scoring script.

In [1]:
import pickle
import numpy as np
import re
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt;
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pom_NEW import *
import os
import sys
import gcsfs
sys.path.append(os.path.abspath("/home/jupyter/POM-feature-drift"))
from google.cloud import storage
from google.cloud import bigquery
import project_config as pc
import common_variables as cv

In [2]:
table_id = 'offer_bridging_eoo_base'
bucket_location = 'EU'
bucket_id       = 'gs://'+pc.bucket+'/pom_scoring'
file_name       = 'eoo_base_2023-01-05'
file_format     = 'CSV'
gcs_file_path   = os.path.join(bucket_id,file_name+'_*.csv')
prefix_name = 'pom_scoring'

In [3]:
#Connection to BQ
client = bigquery.Client(project=pc.project_id) #;
query = """ SELECT column_name, data_type
FROM """+pc.target_dataset+""".INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'offer_bridging_eoo_base'
AND data_type = 'DATE'
"""

date_cols = client.query(query).to_dataframe().iloc[:, 0].tolist()

In [4]:
#Load columns used in new models - speeds up process as we don't need to keep unused columns
cols = {'Account_Number', 'Earliest_Offer_Week', 'base_dt', 'eoo_base_obs_dt', 'Cohort', 'Rack_Rate', 'Ttl_Offer_Discount', 'Customer_Type', 'Country'}
for customer_type in ['DTV']:
    for country in ['UK']:
        for target_type in ['arpu', 'churn', 'ta']:
            for model_type in ['NT', 'L', 'M', 'H']:
                pickle_name = f'pickle_files/{customer_type}_{country}_{target_type}_{model_type}.pkl'
                if os.path.isfile(pickle_name):
                    with open(pickle_name, 'rb') as pickle_file:
                        model = pickle.load(pickle_file)
                    if (type(model) is XGBRegressor) or (type(model) is XGBClassifier):
                        model_columns = model.get_booster().feature_names
                    elif (type(model) is LGBMRegressor) or (type(model) is LGBMClassifier):
                        model_columns = model.feature_name_
                    elif (type(model) is CombinedModel):
                        model_columns = model.feature_name_
                    else:
                        print(f'MODEL TYPE NOT MATCHED {pickle_name}')
                        model_columns = []
                    if model_columns is None:
                        print(f'NONE COLUMNS {pickle_name}')
                        model_columns = []
                    cols = cols.union(model_columns)
rename_andrew = fix_columns()
cols = cols.union({x for x, y in rename_andrew.items() if y in cols})
cols = cols.union({x.rsplit('_', 1)[0] for x in cols})

In [5]:
#Load scoring data from GCS to pandas dataframe
df_main = read_pomdata_to_score(project = pc.project_id, bucket_name = pc.bucket, prefix_name = prefix_name)

blobs are ['pom-etl-process/pom_scoring/eoo_base_2023-01-06_000000000000.csv', 'pom-etl-process/pom_scoring/eoo_base_2023-01-06_000000000001.csv', 'pom-etl-process/pom_scoring/eoo_base_2023-01-06_000000000002.csv', 'pom-etl-process/pom_scoring/eoo_base_2023-01-06_000000000003.csv', 'pom-etl-process/pom_scoring/eoo_base_2023-01-06_000000000004.csv', 'pom-etl-process/pom_scoring/eoo_base_2023-01-06_000000000005.csv']
Total # of date variables are 167
Inital dataframe contains 423212 rows and 987 columns
Shape of the dataset is 423212 rows and 987 columns


## Clean EOO data

In [6]:
dpp = DataPreProcess(df=df_main.rename(columns={'EOO_Base_Obs_Dt' : 'eoo_base_obs_dt'}), cols=cols, date_cols=date_cols)

eoo_base_obs_dt available in the date df


In [7]:
del df_main

In [8]:
_ = dpp.process_dates()

In [9]:
_, _, _, _ = dpp.fill_missing()

In [10]:
_ = dpp.scale_numeric(excl_cols=['Account_Number', 'Earliest_Offer_Week', 'Movies_Active', 'Sports_Active', 'SGE_Active', 'HD_Active', 'MS_Active'])

In [11]:
_ = dpp.one_hot_encode(nunique=100)

[]


In [12]:
data_consolidated = dpp.concat_data(column_fix={'HD_Product_Holding_nan' : 'HD_Product_Holding_None'})
data_consolidated_scaled = dpp.concat_data(scale_numeric=True, column_fix=rename_andrew)

Data contains the total 423212 rows and 264 columns
Data contains the total 423212 rows and 264 columns


In [13]:
data_consolidated_uk_dtv = data_consolidated.loc[(data_consolidated['Country_UK'] == 1) & (data_consolidated['Customer_Type'] == 'DTV')]
data_consolidated_uk_dtv_scaled = data_consolidated_scaled.loc[(data_consolidated_scaled['Country_UK'] == 1) & (data_consolidated_scaled['Customer_Type'] == 'DTV')]

In [14]:
stepup_models = ['NT', 'L', 'M', 'H']

## Score TA Models

In [15]:
data_dict_uk_dtv_stepup = score_data(df=data_consolidated_uk_dtv_scaled, customer_type='DTV', country='UK', target='target_ta', model_types=stepup_models)

************************Model file DTV_UK_ta_NT exists************************
************************Scored data for model DTV_UK_ta_NT************************
************************Model file DTV_UK_ta_L exists************************
************************Scored data for model DTV_UK_ta_L************************
************************Model file DTV_UK_ta_M exists************************
************************Scored data for model DTV_UK_ta_M************************
************************Model file DTV_UK_ta_H exists************************
************************Scored data for model DTV_UK_ta_H************************


In [16]:
scored_df_ta_uk_dtv_new = pd.concat(list(data_dict_uk_dtv_stepup.values()), axis=1)
scored_df_ta = pd.concat([scored_df_ta_uk_dtv_new])

In [17]:
del scored_df_ta_uk_dtv_new

In [18]:
data_consolidated_uk_dtv_scaled.head()

,Earliest_Offer_Week,Lima_ICD_Flag,Account_Number,Rack_Rate,Offer_Discount_SoD,Ttl_Offer_Discount,ARPU,Prev_Offer_Amount_LR,Curr_Offer_Amount_LR,Curr_Offer_Amount_HD,Offers_Applied_Lst_36M_HD,Curr_Offer_Amount_HD_Pack,Curr_Offer_Amount_SKY_BOX_SETS,Curr_Offer_Amount_SKY_KIDS,Contract_Offers_Applied_Lst_12M_DTV,Curr_Contract_Offer_Amount_BB,Curr_Promo_Offer_Amount_DTV,DTV_Activations_In_Last_3Yr,BB_Subscriber_Activations_In_Last_3Yr,BB_Churns_In_Last_3Yr,MS_Active,HD_Active,SGE_Active,TAs_in_last_24m,Age,Prev_Offer_Amount_DTV,Curr_Offer_Amount_DTV,Offers_Applied_Lst_24M_DTV,Prev_Offer_Amount_BB,Prev_Offer_Amount_Movies,Curr_Offer_Amount_Movies,Sports_Active,Sports_Product_Count,Movies_Active,OD_DLs_Completed_In_Last_7d,OD_DLs_Completed_In_Last_30d,OD_DLs_Completed_In_Last_60d,Sky_Consumer_Market_Share,Virgin_Consumer_Market_Share,Throughput_Speed,max_speed_uplift,cancels_3mth,cancels_6mth,cancels_9mth,cancels_12mth,mobile_3mth,mobile_6mth,mobile_9mth,bill_6mth,signin_6mth,vip_6mth,vip_12mth,help_6mth,total_viewing_duration_overlap_3m,total_viewing_duration_overlap_2m,total_viewing_duration_overlap_1m,linear_viewing_total_overlap_1m,linear_viewing_total_overlap_3m,Last_Offer_Applied_Dt_HD_monthdiff,Last_Contract_Offer_Applied_Dt_DTV_monthdiff,Last_Promo_Offer_Applied_Dt_BB_monthdiff,DTV_Last_Activation_Dt_monthdiff,BB_Last_Activation_Dt_monthdiff,BB_Last_Churn_Dt_monthdiff,DTV_Last_PC_Effective_To_Dt_monthdiff,DTV_Last_Active_Block_Dt_monthdiff,DTV_Last_AB_Effective_To_Dt_monthdiff,last_TA_dt_monthdiff,Last_Value_Call_Dt_monthdiff,Last_Platform_Retention_Call_Dt_monthdiff,Last_Service_Call_Dt_monthdiff,Last_All_Call_Dt_monthdiff,Curr_Offer_Start_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_BB_monthdiff,Last_Completed_OD_DL_Dt_monthdiff,Last_Credit_Dt_monthdiff,Customer_Type,BB_Product_Holding_12GB,BB_Product_Holding_BroadbandEssential20ADSL,BB_Product_Holding_BroadbandSuperfast40FTTC,BB_Product_Holding_BroadbandSuperfast80FTTC,BB_Product_Holding_Connect,BB_Product_Holding_ConnectLiteROILegacy,BB_Product_Holding_ConnectUnlimitedROILegacy,BB_Product_Holding_ConnectUnlimitedROI,BB_Product_Holding_EssentialROI,BB_Product_Holding_EssentialPlus,BB_Product_Holding_Everyday,BB_Product_Holding_FibreROILegacy,BB_Product_Holding_FibreLite,BB_Product_Holding_FibreMax,BB_Product_Holding_FibreUltra350Mb,BB_Product_Holding_FibreUnlimitedROILegacy,BB_Product_Holding_FibreUnlimitedPlus,BB_Product_Holding_FibreUnlimitedPro,BB_Product_Holding_FibreUnlimitedProFTTC,BB_Product_Holding_FibreUnlimitedProFTTP,BB_Product_Holding_GigafastFTTP,BB_Product_Holding_GigafastROI,BB_Product_Holding_SKYFibreMaxSOGEA,BB_Product_Holding_SKYFibreUnlimitedSOGEA,BB_Product_Holding_SkyBroadbandLite,BB_Product_Holding_SkyBroadbandLiteROILegacy,BB_Product_Holding_SkyFibre,BB_Product_Holding_SkySuperfastSFTC,BB_Product_Holding_SkyUltrafastMaxROI,BB_Product_Holding_SkyUltrafastPlusROI,BB_Product_Holding_SuperfastFTTP,BB_Product_Holding_SuperfastROI,BB_Product_Holding_Superfast35FTTC,BB_Product_Holding_Superfast35FTTP,BB_Product_Holding_Superfast35SOGEA,BB_Product_Holding_UltrafastROI,BB_Product_Holding_Ultrafast1FTTP,BB_Product_Holding_Ultrafast160SOGfast,BB_Product_Holding_UltrafastPlusFTTP,BB_Product_Holding_UltrafastPlusROI,BB_Product_Holding_Unlimited,BB_Product_Holding_UnlimitedLegacy,BB_Product_Holding_UnlimitedROILegacy,BB_Product_Holding_UnlimitedFibre,BB_Product_Holding_UnlimitedPro,BB_Product_Holding_nan,Talk_Product_Holding_Anytime,Talk_Product_Holding_AnytimeROI,Talk_Product_Holding_Freetime,Talk_Product_Holding_FreetimeROI,Talk_Product_Holding_Other,Talk_Product_Holding_SkyPayAsYouTalk,Talk_Product_Holding_SkyTalkAnytimeExtra,Talk_Product_Holding_SkyTalkAnytimeExtra247,Talk_Product_Holding_SkyTalkAnytimeExtraAnytimeUK,Talk_Product_Holding_SkyTalkAnytimeExtraAnytime,Talk_Product_Holding_SkyTalkAnytimeExtraVoIP,Talk_Product_Holding_SkyTalkEveningsandWeekendsExtra,Talk_Product_Holding_SkyTalkInternationalExtra,Talk_Product_Holding_W

## NannyML Performance Estimation Implementation

### Setup TA Data

In [19]:
ta_data = data_consolidated_uk_dtv_scaled.join(scored_df_ta, on = 'Account_Number')

In [20]:
ta_data.rename(columns = {'pred_ta_NT': 'pred_proba_ta_NT', 'pred_ta_L': 'pred_proba_ta_L', 'pred_ta_M': 'pred_proba_ta_M', 'pred_ta_H': 'pred_proba_ta_H'}, inplace = True)
for treatment in stepup_models:   
    threshold = ta_data[f'pred_proba_ta_{treatment}'].mean()
    pred = (ta_data[f'pred_proba_ta_{treatment}'] >= threshold).astype(int).to_list()
    ta_data[f'pred_ta_{treatment}'] = pred

In [21]:
dates = ta_data.Earliest_Offer_Week.to_list()
timestamps = []
for i, date in enumerate(dates):
    timestamps.append(pd.to_datetime(str(dates[i]) + '-0', format = '%Y%W-%w'))

In [22]:
ta_data['timestamp'] = timestamps

In [23]:
ta_data.sort_values(by = 'timestamp', inplace = True)

In [24]:
ta_model_dict = {}
for treatment in stepup_models:
    with open(f'pickle_files/DTV_UK_ta_{treatment}.pkl', 'rb') as pkl_file:
        ta_model_dict[treatment] = pickle.load(pkl_file)

### Fit and run performance estimation and pickle outputs

In [25]:
import nannyml as nml
# from evidently.report import Report
# from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset
# from evidently import ColumnMapping
from evidently.analyzers.stattests.jensenshannon import jensenshannon_stat_test
from evidently.analyzers.stattests.wasserstein_distance_norm import wasserstein_stat_test
import json

In [26]:
for ref, treatment in zip(['nt', 'low', 'medium', 'high'], stepup_models):   
    ref_data = pd.read_csv(f'data/ta_{ref}_reference_full.csv', low_memory = False)
    ref_data['timestamp'] = pd.to_datetime(ref_data['timestamp'])
    ref_data.sort_values(by = 'timestamp', inplace = True)
    features = ta_model_dict[treatment].feature_name_

    estimator = nml.CBPE(y_pred_proba = 'pred_proba_ta',
                         y_pred = 'pred_ta',
                         y_true = 'target_ta',
                         metrics = ['roc_auc'],
                         timestamp_column_name = 'timestamp',
                         # chunk_period = 'W',
                         chunk_number = 10,
                         problem_type = 'classification_binary'
                        )

    estimator = estimator.fit(ref_data[features + ['pred_proba_ta', 'pred_ta', 'target_ta', 'timestamp']])
    estimated_performance = estimator.estimate(ta_data.rename(columns = {f'pred_proba_ta_{treatment}': 'pred_proba_ta', f'pred_ta_{treatment}': 'pred_ta'})[features + ['pred_proba_ta', 'pred_ta', 'timestamp']])
    
    cohort = file_name.split('_')[2]
    with open(f'data/NannyML_results/ta_{ref}_perf_est_results_{cohort}.pkl', 'wb') as pkl_file:
        pickle.dump(estimated_performance, pkl_file, protocol = pickle.HIGHEST_PROTOCOL)

In [27]:
# A DataFrame containing mean ROC_AUC for each model. In this test we will display this DataFrame, but in production we could export this as a
# BigQuery table for reference while also triggering another phase (such as data drift calcualtions) based on alerts (requires threshold tests).
# NB: Will use the performance estimation on the reference data as a baseline, this is because the algorithm is good at detecting changes
#     but it is hard to know how accurate it is compared to the realised value. This can be investigated further on new data.

perf_est_results = pd.DataFrame(columns = ['EOO_Base', 'Model', 'Mean_Reference_ROC_AUC', 'Mean_Analysis_ROC_AUC', 'Percentage_Decrease', 'Alert'])
perf_est_results = perf_est_results.assign(Model = stepup_models)
perf_est_results = perf_est_results.assign(EOO_Base = file_name.split('_')[2])

threshold = 0.68

cohort = file_name.split('_')[2]
for i, model in enumerate(['nt', 'low', 'medium', 'high']):
    with open(f'data/NannyML_results/ta_{model}_perf_est_results_{cohort}.pkl', 'rb') as pkl_file:
        results = pickle.load(pkl_file)
        results_df = results.to_df(multilevel = False)
        
    perf_est_results.iloc[i, 2] = results_df[results_df.chunk_period == 'reference'].roc_auc_value.mean()
    perf_est_results.iloc[i, 3] = results_df[results_df.chunk_period == 'analysis'].roc_auc_value.mean()
    perf_est_results.iloc[i, 4] = ((perf_est_results.iloc[i, 2] - perf_est_results.iloc[i, 3]) / perf_est_results.iloc[i, 2]) * 100
    perf_est_results.iloc[i, 5] = perf_est_results.iloc[i, 3] < threshold

In [28]:
perf_est_results

,EOO_Base,Model,Mean_Reference_ROC_AUC,Mean_Analysis_ROC_AUC,Percentage_Decrease,Alert
0,2023-01-05,NT,0.754522,0.675518,10.470661,True
1,2023-01-05,L,0.742374,0.640636,13.704425,True
2,2023-01-05,M,0.744044,0.665589,10.544433,True
3,2023-01-05,H,0.741983,0.621938,16.178927,True


### Creation of BQ table containing drift scores and performance estimation
Following from work in data_drift_dashboard.ipynb, the purpose of this table is to be used to create dashboard plots that allow us to see each features/models drift/estimated performance over time.

In [31]:
ta_data2 = ta_data.fillna(0)
dashboard_df = pd.DataFrame()

for treatment, model in zip(stepup_models, ['nt', 'low', 'medium', 'high']):
    
    features = ta_model_dict[treatment].feature_name_
    cat_features = [feature for feature in features if feature.lower() != feature or feature.split('_')[-1] == 'active']
    num_features = [feature for feature in features if feature.lower() == feature and feature.split('_')[-1] != 'active']
    
    if model == 'nt':
        treatment_type = 'NT'
    else:
        treatment_type = model.capitalize()
    
    df = pd.DataFrame(data = {'Features': cat_features + num_features, 'Stattest': None, 'Drift_Score': None,
                              'Cohort': pd.to_datetime(file_name.split('_')[2]), 'Treatment': treatment_type, 'Mean_Estimated_ROC_AUC': None})
    
    # Read in scored training data as referenece
    ref = pd.read_csv(f'gs://andrew-pom/Revenue/ta_step_up_{model}_v2_train_scored.csv', low_memory = False)
    
    ref.drop(columns = ['ind', 'xgb_preds', 'xgb_proba', 'logr_preds', 'logr_proba'], inplace = True)
    ref.rename(columns = {'lgbm_preds': 'pred_ta', 'lgbm_proba': 'pred_proba_ta'}, inplace = True)
    
    for i, feature in enumerate(cat_features):
        df.iloc[i, 1] = 'Jensen-Shannon distance'
        df.iloc[i, 2] = jensenshannon_stat_test(ref[feature], ta_data2[feature], feature_type = 'cat', threshold = 0.1).drift_score
    
    for i, feature in enumerate(num_features):
        i = i + len(cat_features)
        df.iloc[i, 1] = 'Wasserstein distance (normed)'
        df.iloc[i, 2] = wasserstein_stat_test(ref[feature], ta_data2[feature], feature_type = 'num', threshold = 0.1).drift_score
        
    with open(f'data/NannyML_results/ta_{model}_perf_est_results_{cohort}.pkl', 'rb') as pkl_file:
        results = pickle.load(pkl_file).to_df(multilevel = False)
    
    df['Mean_Estimated_ROC_AUC'] = results[results['chunk_period'] == 'analysis'].roc_auc_value.mean()
    
    dashboard_df = pd.concat([dashboard_df, df])   
    # cohort = file_name.split('_')[2]
    # drift_report = Report(metrics = [DataDriftPreset()])
    # drift_report.run(reference_data = ref[features], current_data = ta_data[features])
    # drift_report.save_json(f'drift_reports/ta_{model}_scoring_{cohort}.json')
dashboard_df.reset_index(inplace = True, drop = True)

In [32]:
dashboard_df.reset_index(inplace = True, drop = True)
dashboard_df

,Features,Stattest,Drift_Score,Cohort,Treatment,Mean_Estimated_ROC_AUC
0,hd_active,Jensen-Shannon distance,0.832555,2023-01-05,NT,0.675518
1,sge_product_holding_SGEPaid,Jensen-Shannon distance,0.058957,2023-01-05,NT,0.675518
2,sports_active,Jensen-Shannon distance,0.832555,2023-01-05,NT,0.675518
3,last_ta_reason_FinancialSituation,Jensen-Shannon distance,0.013703,2023-01-05,NT,0.675518
4,talk_product_holding_SkyPayAsYouTalk,Jensen-Shannon distance,0.07928,2023-01-05,NT,0.675518
5,last_ta_outcome_TurnaroundSaved,Jensen-Shannon distance,0.02043,2023-01-05,NT,0.675518
6,ms_active,Jensen-Shannon distance,0.832555,2023-01-05,NT,0.675518
7,hd_product_holding_HDBasicSkyHD,Jensen-Shannon distance,0.294038,2023-01-05,NT,0.675518
8,arpu,Wasserstein distance (normed),0.301114,2023-01-05,NT,0.675518
9,last_ta_dt_monthdiff,Wasserstein distance (normed),17.127308,2023-01-05,NT,0.675518


In [30]:
# dashboard_df = drift_df.copy()

# for treatment, model in zip(stepup_models, ['nt', 'low', 'medium', 'high']):
#     cohort = file_name.split('_')[2]
#     with open(f'drift_reports/ta_{model}_scoring_{cohort}.json', 'r') as file:
#             report = json.load(file)
#     features = ta_model_dict[treatment].feature_name_
    
#     if model == 'nt':
#         treatment_type = 'NT'
#     else:
#         treatment_type = model.capitalize()
        
#     df = pd.DataFrame(data = {'Feature': features, 'Stattest': None, 'Drift_Score': None, 'Cohort': pd.to_datetime(file_name.split('_')[2]), 'Treatment': treatment_type, 'Mean_Estimated_ROC_AUC': None})
    
#     for i, feature in enumerate(features):
#         df.iloc[i, 1] = report['metrics'][1]['result']['drift_by_columns'][feature]['stattest_name']
#         df.iloc[i, 2] = report['metrics'][1]['result']['drift_by_columns'][feature]['drift_score']
    
#     with open(f'data/NannyML_results/ta_{model}_perf_est_results_{cohort}.pkl', 'rb') as pkl_file:
#         results = pickle.load(pkl_file).to_df(multilevel = False)
    
#     df['Mean_Estimated_ROC_AUC'] = results[results['chunk_period'] == 'analysis'].roc_auc_value.mean()
    
#     dashboard_df = pd.concat([dashboard_df, df])
# dashboard_df.reset_index(inplace = True, drop = True)

In [31]:
dashboard_df.Mean_Estimated_ROC_AUC = dashboard_df.Mean_Estimated_ROC_AUC.astype('float')
dashboard_df.Drift_Score = dashboard_df.Drift_Score.astype('float')
dashboard_df.Cohort = dashboard_df.Cohort.astype('str')

dashboard_df.to_gbq('RHS23.POM_ta_data_drift_test', project_id = 'skyuk-uk-decis-models-01-dev', if_exists = 'append')

100%|██████████| 1/1 [00:00<00:00, 6307.22it/s]


## Score ARPU Models

In [15]:
data_dict_uk_dtv_stepup = score_data(df=data_consolidated_uk_dtv_scaled, customer_type='DTV', country='UK', target='target_arpu', model_types=stepup_models)

************************Model file DTV_UK_arpu_NT exists************************
************************Scored data for model DTV_UK_arpu_NT************************
************************Model file DTV_UK_arpu_L exists************************
************************Scored data for model DTV_UK_arpu_L************************
************************Model file DTV_UK_arpu_M exists************************
************************Scored data for model DTV_UK_arpu_M************************
************************Model file DTV_UK_arpu_H exists************************
************************Scored data for model DTV_UK_arpu_H************************


In [16]:
scored_df_arpu_uk_dtv_new = pd.concat(list(data_dict_uk_dtv_stepup.values()), axis=1)
scored_df_arpu = pd.concat([scored_df_arpu_uk_dtv_new])

In [17]:
del scored_df_arpu_uk_dtv_new

## NannyML Performance Estimation Implementation

### Setup ARPU Data

In [18]:
arpu_data = data_consolidated_uk_dtv_scaled.join(scored_df_arpu, on = 'Account_Number')

In [82]:
arpu_data.head()

,Earliest_Offer_Week,Lima_ICD_Flag,Account_Number,Rack_Rate,Offer_Discount_SoD,Ttl_Offer_Discount,ARPU,Prev_Offer_Amount_LR,Curr_Offer_Amount_LR,Curr_Offer_Amount_HD,Offers_Applied_Lst_36M_HD,Curr_Offer_Amount_HD_Pack,Curr_Offer_Amount_SKY_BOX_SETS,Curr_Offer_Amount_SKY_KIDS,Contract_Offers_Applied_Lst_12M_DTV,Curr_Contract_Offer_Amount_BB,Curr_Promo_Offer_Amount_DTV,DTV_Activations_In_Last_3Yr,BB_Subscriber_Activations_In_Last_3Yr,BB_Churns_In_Last_3Yr,MS_Active,HD_Active,SGE_Active,TAs_in_last_24m,Age,Prev_Offer_Amount_DTV,Curr_Offer_Amount_DTV,Offers_Applied_Lst_24M_DTV,Prev_Offer_Amount_BB,Prev_Offer_Amount_Movies,Curr_Offer_Amount_Movies,Sports_Active,Sports_Product_Count,Movies_Active,OD_DLs_Completed_In_Last_7d,OD_DLs_Completed_In_Last_30d,OD_DLs_Completed_In_Last_60d,Sky_Consumer_Market_Share,Virgin_Consumer_Market_Share,Throughput_Speed,max_speed_uplift,cancels_3mth,cancels_6mth,cancels_9mth,cancels_12mth,mobile_3mth,mobile_6mth,mobile_9mth,bill_6mth,signin_6mth,vip_6mth,vip_12mth,help_6mth,total_viewing_duration_overlap_3m,total_viewing_duration_overlap_2m,total_viewing_duration_overlap_1m,linear_viewing_total_overlap_1m,linear_viewing_total_overlap_3m,Last_Offer_Applied_Dt_HD_monthdiff,Last_Contract_Offer_Applied_Dt_DTV_monthdiff,Last_Promo_Offer_Applied_Dt_BB_monthdiff,DTV_Last_Activation_Dt_monthdiff,BB_Last_Activation_Dt_monthdiff,BB_Last_Churn_Dt_monthdiff,DTV_Last_PC_Effective_To_Dt_monthdiff,DTV_Last_Active_Block_Dt_monthdiff,DTV_Last_AB_Effective_To_Dt_monthdiff,last_TA_dt_monthdiff,Last_Value_Call_Dt_monthdiff,Last_Platform_Retention_Call_Dt_monthdiff,Last_Service_Call_Dt_monthdiff,Last_All_Call_Dt_monthdiff,Curr_Offer_Start_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_BB_monthdiff,Last_Completed_OD_DL_Dt_monthdiff,Last_Credit_Dt_monthdiff,Customer_Type,BB_Product_Holding_12GB,BB_Product_Holding_BroadbandEssential20ADSL,BB_Product_Holding_BroadbandSuperfast40FTTC,BB_Product_Holding_BroadbandSuperfast80FTTC,BB_Product_Holding_Connect,BB_Product_Holding_ConnectLiteROILegacy,BB_Product_Holding_ConnectUnlimitedROILegacy,BB_Product_Holding_ConnectUnlimitedROI,BB_Product_Holding_EssentialROI,BB_Product_Holding_EssentialPlus,BB_Product_Holding_Everyday,BB_Product_Holding_FibreROILegacy,BB_Product_Holding_FibreLite,BB_Product_Holding_FibreMax,BB_Product_Holding_FibreUltra350Mb,BB_Product_Holding_FibreUnlimitedROILegacy,BB_Product_Holding_FibreUnlimitedPlus,BB_Product_Holding_FibreUnlimitedPro,BB_Product_Holding_FibreUnlimitedProFTTC,BB_Product_Holding_FibreUnlimitedProFTTP,BB_Product_Holding_GigafastFTTP,BB_Product_Holding_GigafastROI,BB_Product_Holding_SKYFibreMaxSOGEA,BB_Product_Holding_SKYFibreUnlimitedSOGEA,BB_Product_Holding_SkyBroadbandLite,BB_Product_Holding_SkyBroadbandLiteROILegacy,BB_Product_Holding_SkyFibre,BB_Product_Holding_SkySuperfastSFTC,BB_Product_Holding_SkyUltrafastMaxROI,BB_Product_Holding_SkyUltrafastPlusROI,BB_Product_Holding_SuperfastFTTP,BB_Product_Holding_SuperfastROI,BB_Product_Holding_Superfast35FTTC,BB_Product_Holding_Superfast35FTTP,BB_Product_Holding_Superfast35SOGEA,BB_Product_Holding_UltrafastROI,BB_Product_Holding_Ultrafast1FTTP,BB_Product_Holding_Ultrafast160SOGfast,BB_Product_Holding_UltrafastPlusFTTP,BB_Product_Holding_UltrafastPlusROI,BB_Product_Holding_Unlimited,BB_Product_Holding_UnlimitedLegacy,BB_Product_Holding_UnlimitedROILegacy,BB_Product_Holding_UnlimitedFibre,BB_Product_Holding_UnlimitedPro,BB_Product_Holding_nan,Talk_Product_Holding_Anytime,Talk_Product_Holding_AnytimeROI,Talk_Product_Holding_Freetime,Talk_Product_Holding_FreetimeROI,Talk_Product_Holding_Other,Talk_Product_Holding_SkyPayAsYouTalk,Talk_Product_Holding_SkyTalkAnytimeExtra,Talk_Product_Holding_SkyTalkAnytimeExtra247,Talk_Product_Holding_SkyTalkAnytimeExtraAnytimeUK,Talk_Product_Holding_SkyTalkAnytimeExtraAnytime,Talk_Product_Holding_SkyTalkAnytimeExtraVoIP,Talk_Product_Holding_SkyTalkEveningsandWeekendsExtra,Talk_Product_Holding_SkyTalkInternationalExtra,Talk_Product_Holding_W

In [19]:
dates = arpu_data.Earliest_Offer_Week.to_list()
timestamps = []
for i, date in enumerate(dates):
    timestamps.append(pd.to_datetime(str(dates[i]) + '-0', format = '%Y%W-%w'))

In [20]:
arpu_data['timestamp'] = timestamps

In [21]:
arpu_data.sort_values(by = 'timestamp', inplace = True)

In [22]:
arpu_model_dict = {}
for treatment in stepup_models:
    with open(f'pickle_files/DTV_UK_arpu_{treatment}.pkl', 'rb') as pkl_file:
        arpu_model_dict[treatment] = pickle.load(pkl_file)

### Fit and run performance estimation and pickle outputs

In [23]:
import nannyml as nml
from evidently.analyzers.stattests.jensenshannon import jensenshannon_stat_test
from evidently.analyzers.stattests.wasserstein_distance_norm import wasserstein_stat_test

In [25]:
for ref, treatment in zip(['nt', 'low', 'medium', 'high'], stepup_models):   
    ref_data = pd.read_csv(f'data/arpu_{ref}_reference_full.csv', low_memory = False)
    ref_data['timestamp'] = pd.to_datetime(ref_data['timestamp'])
    ref_data.sort_values(by = 'timestamp', inplace = True)
    features = arpu_model_dict[treatment].feature_name_

    estimator = nml.DLE( feature_column_names = features,
                         y_pred = 'pred_arpu',
                         y_true = 'target_arpu',
                         metrics = ['rmse', 'mae'],
                         timestamp_column_name = 'timestamp',
                         chunk_period = 'W',
                         # chunk_number = 10,
                        )

    estimator = estimator.fit(ref_data[features + ['pred_arpu', 'target_arpu', 'timestamp']])
    estimated_performance = estimator.estimate(arpu_data.rename(columns = {f'pred_arpu_{treatment}': 'pred_arpu'})[features + ['pred_arpu', 'timestamp']])
    
    cohort = file_name.split('_')[2]
    with open(f'data/NannyML_results/arpu_{ref}_perf_est_results_{cohort}.pkl', 'wb') as pkl_file:
        pickle.dump(estimated_performance, pkl_file, protocol = pickle.HIGHEST_PROTOCOL)

In [26]:
# A DataFrame containing mean RMSE and MAE for each model. In this test we will display this DataFrame, but in production we could export this as a
# BigQuery table for reference while also triggering another phase (such as data drift calcualtions) based on alerts (requires threshold tests).
# NB: Will use the performance estimation on the reference data as a baseline, this is because the algorithm is good at detecting changes
#     but it is hard to know how accurate it is compared to the realised value. This can be investigated further on new data.

perf_est_results = pd.DataFrame(columns = ['EOO_Base', 'Model', 'Mean_Reference_RMSE', 'Mean_Analysis_RMSE', 'Percentage_Increase_RMSE',
                                           'Mean_Reference_MAE', 'Mean_Analysis_MAE', 'Percentage_Increase_MAE', 'Alert'])
perf_est_results = perf_est_results.assign(Model = stepup_models)
perf_est_results = perf_est_results.assign(EOO_Base = file_name.split('_')[2])

threshold = 70

cohort = file_name.split('_')[2]
for i, model in enumerate(['nt', 'low', 'medium', 'high']):
    with open(f'data/NannyML_results/arpu_{model}_perf_est_results_{cohort}.pkl', 'rb') as pkl_file:
        results = pickle.load(pkl_file)
        results_df = results.to_df(multilevel = False)
        
    perf_est_results.iloc[i, 2] = results_df[results_df.chunk_period == 'reference'].rmse_value.mean()
    perf_est_results.iloc[i, 3] = results_df[results_df.chunk_period == 'analysis'].rmse_value.mean()
    perf_est_results.iloc[i, 4] = ((perf_est_results.iloc[i, 3] - perf_est_results.iloc[i, 2]) / perf_est_results.iloc[i, 2]) * 100
    
    perf_est_results.iloc[i, 5] = results_df[results_df.chunk_period == 'reference'].mae_value.mean()
    perf_est_results.iloc[i, 6] = results_df[results_df.chunk_period == 'analysis'].mae_value.mean()
    perf_est_results.iloc[i, 7] = ((perf_est_results.iloc[i, 6] - perf_est_results.iloc[i, 5]) / perf_est_results.iloc[i, 5]) * 100
    
    perf_est_results.iloc[i, 8] = perf_est_results.iloc[i, 3] > threshold

In [27]:
perf_est_results

,EOO_Base,Model,Mean_Reference_RMSE,Mean_Analysis_RMSE,Percentage_Increase_RMSE,Mean_Reference_MAE,Mean_Analysis_MAE,Percentage_Increase_MAE,Alert
0,2023-01-05,NT,56.889477,77.721394,36.618227,40.938128,59.607274,45.603321,True
1,2023-01-05,L,50.544726,67.58382,33.710924,34.592002,53.391651,54.346808,False
2,2023-01-05,M,49.518905,76.796577,55.08537,32.521126,53.798615,65.426666,True
3,2023-01-05,H,48.431117,67.897941,40.194868,31.811065,47.278717,48.623495,False


### Creation of BQ table containing drift scores and performance estimation
Following from work in data_drift_dashboard.ipynb, the purpose of this table is to be used to create dashboard plots that allow us to see each features/models drift/estimated performance over time.

In [28]:
arpu_data2 = arpu_data.fillna(0)
dashboard_df = pd.DataFrame()

for treatment, model in zip(stepup_models, ['nt', 'low', 'medium', 'high']):
    
    features = arpu_model_dict[treatment].feature_name_
    cat_features = [feature for feature in features if feature.lower() != feature or feature.split('_')[-1] == 'active']
    num_features = [feature for feature in features if feature.lower() == feature and feature.split('_')[-1] != 'active']
    
    if model == 'nt':
        treatment_type = 'NT'
    else:
        treatment_type = model.capitalize()
    
    df = pd.DataFrame(data = {'Features': cat_features + num_features, 'Stattest': None, 'Drift_Score': None,
                              'Cohort': pd.to_datetime(file_name.split('_')[2]), 'Treatment': treatment_type, 'Mean_Estimated_RMSE': None, 'Mean_Estimated_MAE': None})
    
    # Read in scored training data as referenece
    ref = pd.read_csv(f'gs://andrew-pom/Revenue/revenue_step_up_{model}_6m_v3_train_scored.csv', low_memory = False)
    
    ref.drop(columns = ['ind', 'xgbr_preds', 'linr_preds'], inplace = True)
    ref.rename(columns = {'lgbmr_preds': 'pred_arpu'}, inplace = True)
    
    for i, feature in enumerate(cat_features):
        df.iloc[i, 1] = 'Jensen-Shannon distance'
        df.iloc[i, 2] = jensenshannon_stat_test(ref[feature], arpu_data2[feature], feature_type = 'cat', threshold = 0.1).drift_score
    
    for i, feature in enumerate(num_features):
        i = i + len(cat_features)
        df.iloc[i, 1] = 'Wasserstein distance (normed)'
        df.iloc[i, 2] = wasserstein_stat_test(ref[feature], arpu_data2[feature], feature_type = 'num', threshold = 0.1).drift_score
        
    with open(f'data/NannyML_results/arpu_{model}_perf_est_results_{cohort}.pkl', 'rb') as pkl_file:
        results = pickle.load(pkl_file).to_df(multilevel = False)
    
    df['Mean_Estimated_RMSE'] = results[results['chunk_period'] == 'analysis'].rmse_value.mean()
    df['Mean_Estimated_MAE'] = results[results['chunk_period'] == 'analysis'].mae_value.mean()
    
    dashboard_df = pd.concat([dashboard_df, df])   
    # cohort = file_name.split('_')[2]
    # drift_report = Report(metrics = [DataDriftPreset()])
    # drift_report.run(reference_data = ref[features], current_data = ta_data[features])
    # drift_report.save_json(f'drift_reports/ta_{model}_scoring_{cohort}.json')
dashboard_df.reset_index(inplace = True, drop = True)

In [29]:
dashboard_df

,Features,Stattest,Drift_Score,Cohort,Treatment,Mean_Estimated_RMSE,Mean_Estimated_MAE
0,hd_active,Jensen-Shannon distance,0.832555,2023-01-05,NT,77.721394,59.607274
1,sge_product_holding_SGEPaid,Jensen-Shannon distance,0.061397,2023-01-05,NT,77.721394,59.607274
2,curr_offr_sub_type_sports_SPORTS,Jensen-Shannon distance,0.00855,2023-01-05,NT,77.721394,59.607274
3,bb_product_holding_FibreMax,Jensen-Shannon distance,0.238127,2023-01-05,NT,77.721394,59.607274
4,hd_product_holding_HDPremiumRose,Jensen-Shannon distance,0.128843,2023-01-05,NT,77.721394,59.607274
5,hd_product_holding_HDBasicSkyHD,Jensen-Shannon distance,0.298694,2023-01-05,NT,77.721394,59.607274
6,last_ta_outcome_TurnaroundSaved,Jensen-Shannon distance,0.018651,2023-01-05,NT,77.721394,59.607274
7,prev_offr_sub_type_sports_SPORTS,Jensen-Shannon distance,0.143692,2023-01-05,NT,77.721394,59.607274
8,ttl_offer_discount,Wasserstein distance (normed),0.152913,2023-01-05,NT,77.721394,59.607274
9,bill_6mth,Wasserstein distance (normed),0.05558,2023-01-05,NT,77.721394,59.607274


In [30]:
dashboard_df.Mean_Estimated_RMSE = dashboard_df.Mean_Estimated_RMSE.astype('float')
dashboard_df.Mean_Estimated_MAE = dashboard_df.Mean_Estimated_MAE.astype('float')
dashboard_df.Drift_Score = dashboard_df.Drift_Score.astype('float')
dashboard_df.Cohort = dashboard_df.Cohort.astype('str')

dashboard_df.to_gbq('RHS23.POM_arpu_data_drift_test', project_id = 'skyuk-uk-decis-models-01-dev', if_exists = 'append')

100%|██████████| 1/1 [00:00<00:00, 6512.89it/s]
